Import relevant packages

In [3]:
import sys # to get "unresolved" packages
sys.path.append("/Users/a1765262/opt/anaconda3/lib/python3.9/site-packages")

import pandas as pd # for data frames
import os # for working directories
import numpy as np # for numbers
import matplotlib.pyplot as plt # for plotting
from corextopic import corextopic as ct # for topic model
import string # for preprocessing
import sklearn.feature_extraction.text # for vectoriser
from nltk.sentiment import SentimentIntensityAnalyzer # for sentiment
import pickle
import scipy
from nltk import FreqDist
import shifterator as sh
import datetime
# from readability import readability

os.chdir("..")

sys.path.append("./Other")
import irulan

channel_list = ["ABC1", "Ch7", "Ch9", "Ch10", "SBS", "ABC24"]

Test a topic model anchored on 'sport' for different document lengths. Plot the number of words in each document for 5-minute and program lengths. 

In [ ]:
# vector of seconds that we will look at (9999 seconds represents the program split)
seconds = [10, 60, 120, 300, 600, 1200, 1800, 9999]

abc1_2022 = pd.read_csv("./Data/ABC1_2022.csv")

for i, s in enumerate(seconds):

    # clean text so that documents are s seconds long
    text = irulan.clean_split_mins(abc1_2022, s)

    # train topic model
    _, tm = irulan.tm(text, anchors = ['sport'], num_topics= 40)

    print(tm.get_topics()[0])

    # if we have split into 5-minute or program-length intervals, make histogram
    if s in [60, 9999]:
        plt.hist([len(doc.split()) for doc in text], bins = 50)
        plt.title("The number of words contained in each program")
        plt.xlabel("Number of words")
        plt.ylabel("Frequency")

Find the total correlation of topic models with the number of topics adjusted from 10 to 100.

In [ ]:
text = pickle.load(open("./Data/2022_all_clean.pkl", "rb"))

# make the topic model
n_topics = np.array(range(10, 110, 10))
iterations = 30
tcs = np.empty((len(n_topics), iterations))

# initialise vectorizer 
vectorizer = sklearn.feature_extraction.text.CountVectorizer(max_features = 10000, binary = True) 

# get sparse matrix of number of times each word appears in each document
matrix = vectorizer.fit_transform(text)

# get the list of words
words = sorted(vectorizer.vocabulary_.keys())

for k, n in enumerate(n_topics):
    for i in range(iterations):

        # train the topic model
        tm = ct.Corex(n_hidden = n, words = words, max_iter = 400, seed = i) 
        tm.fit(matrix, words = words, anchor_strength = 10)

        tcs[k, i] = tm.tc
        print(f'Value {i} for {n} topics is: {tm.tc}')
        del tm

# make violin plot
plt.violinplot(tcs, positions = np.array(range(10, 110, 10)), widths = 5)
plt.title("The total correlation of topic models on 2022 data adjusting the number of topics")
plt.ylabel("Total correlation")
plt.xlabel("Number of topics")

Pearson and Spearman similarities for topic models with an adjacent number of topics.

Generate unsupervised topic models trained on 2022 data from ABC1.

Train 30 topic models with the seed 'sport'.

Train 30 topic models with anchors for each method of inference and calculate Pearson correlation. 

Compare different percentages to subsample with the ground truth.

Generate unsupervised topic models trained on 2022 data for each channel.

Compare unsupervised topic models trained on each channel with Pearson similarity.

Compare anchored topic models trained on each channel with Pearson similarity.

Create a hierarchical topic model to combine sport topics into one larger overarching sport topic. 

Plot media attention for supervised topic models.

Calculate the coverage bias for some topics.

Get political word counts. 

Calculate political coverage bias.